**MODEL OF MACHINE LEARNING FOR PREDICT CANCELLATION OF SUBSCRIPTIONS (MODEL OF CLASSIFICATION)**

In [ ]:
#importing packages that will be used

In [ ]:
#For Data treatment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
#Libraries Skit Learn
from sklearn.model_selection import train_test_split # Used to separate Data to training and testing
from sklearn.preprocessing import StandardScaler # Used to make a Data Standardization 
from sklearn.preprocessing import LabelEncoder #To make OneHotEncoder
from sklearn.metrics import accuracy_score #Used to assess acuraccy from the model predctive
from sklearn.neighbors import KNeighborsClassifier # The model of Machine Learning
from imblearn import under_sampling, over_sampling # Used to make a balance of Data
from imblearn.over_sampling import SMOTE # Used to make a balance of data too 
# to remove alert notices
import warnings
warnings.filterwarnings('ignore')

#for dataframe display
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

   **1- ANALISYS AND DATA COLLECTION**

In [ ]:
df_original = pd.read_csv("C:/Users/lavin/Documents/dados.csv", sep = ';')

In [ ]:
df_original.shape

**448447 Lines and 24 Columns**

In [ ]:
df_original.head()

In [ ]:
df_original.info()

**7 categorical Columns**

In [ ]:
df_original.describe()

In [ ]:
df_original.isnull().sum()

**2 Columns with values missings** QT_FILHOS and DT_CANCELAMENTO

In [ ]:
df_original.nunique()

In [ ]:
df_original.info()

In [ ]:
start = pd.to_datetime(df_original['DT_AQUISICAO']).dt.date.min()
end = pd.to_datetime(df_original['DT_AQUISICAO']).dt.date.max()
print(f'Period of data - Start: {start} / End: {end}')
                        

**2- ANALYSIS OF CATEGORICAL VARIABLES (ALL OF WHICH ARE OBJECTS)**
- SITUACAO      
- NOME_PRODUTO  
- DURACAO_CONTRATO
- SEXO
- FORMA_AQUISICAO


In [ ]:
df_original.groupby(['SITUACAO']).size()

In [ ]:
df_original.groupby(['NOME_PRODUTO']).size()

In [ ]:
df_original.groupby(['DURACAO_CONTRATO']).size()

In [ ]:
df_original.groupby(['SEXO']).size()

In [ ]:
df_original.groupby(['FORMA_AQUISICAO']).size()

- The product more used is the Basic Plan (30 canais hd)
- And the least used product is the Medium Plan TOP (90 canais hd)
- The limit of duration of contract is 48 months
- The values missings of dataframe is the columns: QT_FILHOS and DT_CANCELAMENTO

**I made these graphics below to better visualize of data**

In [ ]:
plt.rcParams['figure.figsize'] = [12.00,3.50]
plt.rcParams['figure.autolayout'] = True
f, axes = plt.subplots(1,2)
sns.countplot(data = df_original, x='SEXO', hue='SITUACAO', ax = axes [0])
sns.countplot(data = df_original, x='FORMA_AQUISICAO', hue='SITUACAO', ax = axes[1])
plt.show

**In these graphics who we can see above show us a insight valuable:**
- The costummers prefer to make shop online and the diference between cancellations is very small

In [ ]:
plt.rcParams['figure.figsize'] = [7.00,5.00]
plt.rcParams['figure.autolayout'] = True
sns.countplot(data = df_original, x = 'DURACAO_CONTRATO', hue = 'SITUACAO')
plt.show

**Here we can see whats is the contract preferred by the costummers: 48 Months**

In [ ]:
plt.rcParams['figure.figsize'] = [12.00,7.00]
plt.rcParams['figure.autolayout'] = True
sns.countplot(data = df_original, x = 'NOME_PRODUTO', hue = 'SITUACAO')
plt.xticks(rotation = 70)
plt.show

**On column 'NOME_PRODUTO' we can see the favorite product of costummers is the : Plano Basico (30 canais HD)**

**3- ANAlISYS OF VARIABLES NUMERICAL**

**First i used a function 'FOR' to make a variable with all numeric columns**

In [ ]:
# we will not use the column 1 ID_CLIENTE because it is not necessary
numeric_variable = [] #i make this variable empty for put the numeric collumns in there
for i in df_original.columns[1:24].tolist():
         if df_original.dtypes[i] == 'int64' or df_original.dtypes[i] == 'float64':
           print (i, ':', df_original.dtypes[i])
           numeric_variable.append(i)

In [ ]:
len(numeric_variable)

**16 numeric columns**

**Now i make graphics BOXPLOT to check if dataframe have outliers (using numeric_variable)**

In [ ]:
plt.rcParams ['figure.figsize'] = (14.00, 24.00)
plt.rcParams['figure.autolayout'] = True
f, axes = plt.subplots(4,4)

#I created a 'for' function that every time it passes through a column it creates a boxplot graph

line = 0
column = 0

for i in numeric_variable:
    sns.boxplot(data=df_original, y=i, ax=axes[line][column])
    column += 1
    if column == 4:
        line+=1
        column = 0
plt.show()

**Columns whith OUTLIERS:**
- QT_FILHOS 
- QT_PC_PAGAS
- QT_PC_PAGA_EM_DIA
- 

**4- TRATAMENT OF DATA WITH OUTLIERS** 

**I make a tratament of column "QT_FILHOS"**
- I create a new dataframe to put the processed data 
- I put on new dataframe the column QT_FILHOS without outliers
- After this i visualize the median from "QT_FILHOS" column for replace the null values
- Last i make a 'fillna' function for put in null values the median of column QT_FILHOS

In [ ]:
df_data = df_original.loc[df_original['QT_FILHOS'] <= 2]

In [ ]:
df_data.groupby(['QT_FILHOS']).size()

In [ ]:
#visualize the median of the data
df_data['QT_FILHOS'].median()

In [ ]:
#I filled in the null values with the median of the data
df_data['QT_FILHOS']= df_data['QT_FILHOS'].fillna((df_data['QT_FILHOS'].median()))
df_data['QT_FILHOS'].isnull().sum()

In [ ]:
# replacing object data with numeric data
df_data['DURACAO_CONTRATO']= df_data['DURACAO_CONTRATO'].replace(['12 Meses'],12)
df_data['DURACAO_CONTRATO']= df_data['DURACAO_CONTRATO'].replace(['24 Meses'],24)
df_data['DURACAO_CONTRATO']= df_data['DURACAO_CONTRATO'].replace(['36 Meses'],36)
df_data['DURACAO_CONTRATO']= df_data['DURACAO_CONTRATO'].replace(['48 Meses'],48)

In [ ]:
df_data.head()

**Doing tratament of column QT_PC_PAGAS**
- In this part we can see a error between 'DURACAO_CONTRATO', 'QT_PC_PAGAS' and 'QT_PC_PAGA_EM_DIA'
- Whats is the error? in the 'DURACAO_CONTRATO' column we only have contracts of up to 48 months, but in the 'QT_PC_PAGAS' and 'QT_PC_PAGA_EM_DIA' columns we have installments with values greater than 48 months
- So i put in dataframe 'df_data' only data above 48 months

In [ ]:
df_data = df_original.loc[df_original['QT_PC_PAGAS'] <= 48]
df_data.groupby(['QT_PC_PAGAS']).size()

**Doing the same thing with 'QT_PC_PAGA_EM_DIA' column what i made with the column above**

In [ ]:
df_data = df_original.loc[df_original['QT_PC_PAGA_EM_DIA'] <= 48]
df_data.groupby(['QT_PC_PAGA_EM_DIA']).size()

**5- FEATURE ENGINEERING**
- I categorize the columns "QT_PC_PAGAS"
- Create a new column called 'NIVEL_PAGAMENTO"
- Used a function of pandas pd.cut for categorize the data

In [ ]:
bins = (-100, 3, 6, 12, 48)
labels = ('BAD', 'MEDIUM', 'GOOD', 'GREAT')

df_data['NIVEL_PAGAMENTO'] = pd.cut(df_data['QT_PC_PAGAS'], bins = bins, labels = labels)
df_data.groupby(['NIVEL_PAGAMENTO']).size()

**6- TRANSFORMATION OF DATA FOR PREDICTION** 
- First i copied the dataframe df_data to dataframe df_data2 to do this transformation and dont lose the data if something happens
- I transform data using the function Label Encoder of Skit-Learn

In [ ]:
df_data2 = df_data.copy()
df_data2.head()

In [ ]:
#I make another replace and another fillna because the first didn't worked on df_data2
df_data2['DURACAO_CONTRATO']= df_data2['DURACAO_CONTRATO'].replace(['12 Meses'],12)
df_data2['DURACAO_CONTRATO']= df_data2['DURACAO_CONTRATO'].replace(['24 Meses'],24)
df_data2['DURACAO_CONTRATO']= df_data2['DURACAO_CONTRATO'].replace(['36 Meses'],36)
df_data2['DURACAO_CONTRATO']= df_data2['DURACAO_CONTRATO'].replace(['48 Meses'],48)
df_data2.head()

df_data2['QT_FILHOS']= df_data2['QT_FILHOS'].fillna((df_data2["QT_FILHOS"]).mean())
df_data2['QT_FILHOS'].isnull().sum()

In [ ]:
lb = LabelEncoder()

df_data2['SEXO'] = lb.fit_transform(df_data2['SEXO'])
df_data2['FORMA_AQUISICAO'] = lb.fit_transform(df_data2['FORMA_AQUISICAO'])
df_data2['NOME_PRODUTO'] = lb.fit_transform(df_data2['NOME_PRODUTO'])
df_data2['NIVEL_PAGAMENTO'] = lb.fit_transform(df_data2['NIVEL_PAGAMENTO'])

In [ ]:
df_data2.head(20)

In [ ]:
# REMOVING THE COLUMNS THAT WE WILL NOT USE
df_data2.columns.tolist()

In [ ]:
#removing
columns = [ 'FORMA_AQUISICAO',
 'IDADE_CLIENTE',
 'SEXO',
 'QT_FILHOS',
 'DIAS_ATIVO',
 'MESES_ATIVO',
 'DURACAO_CONTRATO',
 'VL_PLANO_ADESAO',
 'VL_PLANO_ATUAL',
 'NOME_PRODUTO',
 'QT_PONTOS_INSTALADOS',
 'QT_PC_PAGAS',
 'QT_PC_VENCIDAS',
 'QT_PC_PAGA_ATRASO',
 'QT_PC_PAGA_EM_DIA',
 'QT_ACORDO_PAGAMENTO',
 'VL_MENSALIDADE_ATRASO',
 'VL_MENSALIDADE_DT_AQUISICAO',
 'VL_MENSALIDADE_DT_ATUAL',
 'NIVEL_PAGAMENTO',
 'COD_SITUACAO']

df_data2 = pd.DataFrame(df_data2, columns = columns)

In [ ]:
df_data2.head()

In [ ]:
#Checking type of data
df_data2.info()

**8- DOING THE BALANCE OF DATA**
- When i made the first graphs at the beginning of the analysis i noticed  that there was an imbalance between the columns
- So i match this data to model make the predction

In [ ]:
#I made a graph for the better visualization
plt.rcParams['figure.figsize'] = (7.00 , 4.00)
plt.rcParams['figure.autolayout'] = True
df_data2.COD_SITUACAO.value_counts().plot(kind = 'bar', title = "Cliente Ativos x Clientes Inativos" )
# 0 = Ativo 
# 1 = Inativo

In [ ]:
# I saparated predictor columns and target column
PREDICTOR = df_data2.iloc[:, 0:20]
TARGET = df_data2.iloc[:,20]

In [ ]:
PREDICTOR.head()

In [ ]:
TARGET.head()

In [ ]:
#making data balancing 

#seed to get the same result 
seed = (100)

#Creating the balancer smote
balance = SMOTE(random_state = seed)

#applying the balancer
PREDICTOR_RES, TARGET_RES = balance.fit_resample(PREDICTOR, TARGET)


In [ ]:
#Now viewing the balanced TARGET table
plt.rcParams['figure.figsize'] = (12.00 , 5.00)
plt.rcParams['figure.autolayout'] = True
TARGET_RES.value_counts().plot(kind = 'bar', title = "Cliente Ativos x Clientes Inativos" )

In [ ]:
#Quantity of data before balancing
PREDICTOR.shape

In [ ]:
#Quantity of data before balancing
TARGET.shape

In [ ]:
#Quantity of data after balancing
TARGET_RES.shape

In [ ]:
#Quantity of data after balancing
PREDICTOR_RES.shape

**10- Training the model**
- Separated data of training and testing
- Data Standardization
- Create a model

In [ ]:
#Training and Testing
x_train, x_test, y_train, y_test = train_test_split(PREDICTOR_RES, TARGET_RES, random_state = 42)

In [ ]:
#Standardization
standardization = StandardScaler()

X_train_standard = standardization.fit_transform(x_train)
X_test_standard = standardization.transform(x_test)

In [ ]:
X_train_standard

In [ ]:
accuracy = []

In [ ]:
#Created a Range for KNN
Kvals = range (3,10,2)

In [ ]:
start = time.time()
for k in Kvals:
    modelKNN = KNeighborsClassifier(n_neighbors = k)
    modelKNN.fit(X_train_standard, y_train)

    score = modelKNN.score(X_test_standard, y_test)
    print(f' with value of: {k}, the accuracy of model is:{score*100}')
    accuracy.append(score)
end = time.time()
print('Model training time:', end - start)    

In [ ]:
#Checking the model with better score
i = np.argmax(accuracy)
print(f'The value k = {Kvals[i]}, have the greater accuracy: {accuracy[i]*100}') 

In [ ]:
#Create a final version of model with greater value of 'k'
ModelFinal = KNeighborsClassifier(n_neighbors = Kvals[i])

In [ ]:
#Model training
ModelFinal.fit(X_train_standard, y_train)

In [ ]:
#Predict with data of test
predictors = ModelFinal.predict(X_test_standard)

In [ ]:
#Model KNN Accuracy
print(f'Accuracy of model:{accuracy_score(y_test, predictors)}')

**This means that out of 100 times the model got it right 97%**